In [1]:
import pandas as pd
df = pd.read_json("ScaleQM+_test.json")

In [3]:
def extract_missing_steps(text):
    pattern = re.compile(
        r'Missing Step (\d+)：\s*'
        r'The missing step should be placed between Step (\d+) and Step (\d+)\.\s*'
        r'The missing step is:\s*(.*?)'
        r'(?=\s*Missing Step \d+：|\Z)',
        re.DOTALL
    )
    matches = pattern.findall(text)
    results = []
    for match in matches:
        a, x, y, z = match
        results.append({
            'a': a.strip(),
            'x': x.strip(),
            'y': y.strip(),
            'z': z.strip()
        })
    return results

def filter_results(results):
    filtered_results = []
    for result in results:
        x = int(result['x'])
        y = int(result['y']) 
        z = result['z']
        if x + 1 == y and not z.startswith('####'):
            filtered_results.append(result)
    return filtered_results

def sort_results_by_x(results):
    sorted_results = sorted(results, key=lambda result: int(result['x']))
    return sorted_results

def process_text(text):
    results = extract_missing_steps(text)
    filtered_results = filter_results(results)
    sorted_results = sort_results_by_x(filtered_results)
    return sorted_results



In [6]:
import re
results = []
for i in range(len(df)):
    data = process_text(df.iloc[i]["messages"][2]["content"])
    
    result = {
        "positions": [(int(item["x"]), int(item["y"])) for item in data],
        "texts": [item["z"] for item in data]
    }
    
    results.append(result)


In [11]:
import pickle
with open('gold.pkl', 'wb') as f:
    pickle.dump(results, f)